In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import shutil
import os
from ftplib import FTP


In [ ]:
data_dir = '/Users/gbromley/data/NGP_Stations/'
#station = 'CA004057120'
station_info_file = '/Users/gbromley/Dropbox/Montana_Climate_Project/NGP_stations.csv'
station_info = pd.read_csv(station_info_file)
#temp_ca = pd.read_csv(data_dir+station+'.csv')

In [ ]:
station_info['Station']

In [ ]:
#test = temp_ca.where(temp_ca['3']=='TMAX')


In [ ]:
#test = test.dropna(axis=0, how='all')


In [ ]:
for index, station in station_info['Station'].iteritems():
    try:
        stat_data = pd.read_csv(data_dir+station+'.csv')
    except Exception:
        print(station +' not found')
        continue
    new_data = np.empty(shape=(len(stat_data['2'])*31,6))
    new_data.fill(np.nan)


    months=[31,28,31,30,31,30,31,31,30,31,30,31]

    day_count = 0
    #test = temp_ca.where(temp_ca['3']=='TMAX')
    #test = test.dropna(axis=0, how='all')
    prev_month = 0
    for index, row in stat_data.iterrows():
    
        month = int(row[3]-1) #accounting for off by 1
        num_days = months[month] #get the standard number of days, ignoring leap year
    
        if prev_month != month:
        
            new_data[day_count:day_count+num_days,0]= row[2]
            new_data[day_count:day_count+num_days,1]= row[3]
            new_data[day_count:day_count+num_days,2]= np.arange(1,num_days+1)
            if index!=0:
                day_count = day_count + num_days
    
        
    
    
        if row[4]=='TMAX':
            new_data[day_count:day_count+num_days,3] = row[5:num_days*4+2:4]/10 # the +2 is to catch the off by 1 error
        
    
        if row[4]=='TMIN':
            new_data[day_count:day_count+num_days,4] = row[5:num_days*4+2:4]/10 # the +2 is to catch the off by 1 error
        
        if row[4]=='PRCP':
            new_data[day_count:day_count+num_days,5] = row[5:num_days*4+2:4]/10 # the +2 is to catch the off by 1 error
        
    
        prev_month=month
    
    
    new_data = new_data[~np.isnan(new_data).any(axis=1)]
    time_data = (new_data[:,0],new_data[:,1],new_data[:,2])
    pd_data = pd.DataFrame(new_data, columns = ['year','month','day','tmax','tmin','precip'])

    pd_data = pd_data.set_index(pd.to_datetime(pd_data[['year', 'month', 'day']]))

    pd_data = pd_data.drop(['year', 'month', 'day'],axis=1)

    pd_data = pd_data.replace(-999.9, value=np.nan)
    
    nc_version = pd_data.to_xarray()

    nc_version.coords['station'] = station



    nc_version.to_netcdf(data_dir+'nc_files/'+station+'.nc', mode='w')
    
    

In [ ]:
new_data

In [ ]:
new_data = new_data[~np.isnan(new_data).any(axis=1)]
time_data = (new_data[:,0],new_data[:,1],new_data[:,2])

In [ ]:
time_data

In [ ]:
test2 = pd.DataFrame(new_data, columns = ['year','month','day','tmax','tmin','precip'])

test2 = test2.set_index(pd.to_datetime(test2[['year', 'month', 'day']]))

test2 = test2.drop(['year', 'month', 'day'],axis=1)

test2 = test2.replace(-999.9, value=np.nan)

In [ ]:

ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login()
ftp.cwd('pub/data/ghcn/daily')
ftp.retrbinary('RETR ghcnd-stations.txt', open('ghcnd-stations.txt', 'wb').write)
ftp.quit()

  # Read in GHCND-D Stations File
ghcnd_stnfile='ghcnd-stations.txt'
ghcnd_stations= np.genfromtxt(ghcnd_stnfile,delimiter=(11,9,10,7,4,30),dtype=str)
station_meta = dict(zip(ghcnd_stations[:,0],ghcnd_stations[:,1:6]))

In [ ]:
station_meta['CA004057120']

In [ ]:
dict(ghcnd_stations[0],ghcnd_stations)

In [ ]:
nc_version = test2.to_xarray()

nc_version.coords['station'] = station



nc_version.to_netcdf(data_dir+station+'.nc')